In [1]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import pandas as pd
import datetime
import pytz
import schedule
import time
from datetime import datetime
from io import StringIO
import os


In [5]:
url = "https://www.zse.co.zw/price-sheet/"

In [6]:
def get_todays_data():
  response = requests.get(url)
  data = pd.read_html(StringIO(response.text))
  df = pd.DataFrame(data[0])
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.dropna()
  return df

def get_open_price():
  df = get_todays_data()
  open_price = df[['Company Name','Opening Price']].T
  open_price.columns  = open_price.iloc[0]
  open_price = open_price[1:]
  open_price.index.name = 'Date'
  open_price = open_price.rename(index={open_price.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return open_price

def get_close_price():
  df = get_todays_data()
  close_price = df[['Company Name','Closing Price']].T
  close_price.columns  = close_price.iloc[0]
  close_price = close_price[1:]
  close_price.index.name = 'Date'
  close_price = close_price.rename(index={close_price.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return close_price

def get_vol_traded():
  df = get_todays_data()
  vol_traded = df[['Company Name','Total Traded Volume']].T
  vol_traded.columns  = vol_traded.iloc[0]
  vol_traded = vol_traded[1:]
  vol_traded.index.name = 'Date'
  vol_traded = vol_traded.rename(index={vol_traded.index[0]: datetime.now(pytz.timezone('Africa/Harare'))})
  return vol_traded

In [7]:
global open_price
global close_price
global vol_traded
open_price = get_open_price()
close_price = get_close_price()
vol_traded = get_vol_traded()

In [8]:
#this is the code that needs to run everyday
def update_data():
  global open_price
  open_price = pd.concat([open_price, get_open_price()], axis=0) # Update the variables
  global close_price
  close_price = pd.concat([close_price, get_close_price()], axis=0)
  global vol_traded
  vol_traded = pd.concat([vol_traded, get_vol_traded()], axis=0)

  return open_price, close_price, vol_traded

update_data()

(Company Name                     Afdis Distillers Limited  \
 Date                                                        
 2025-06-19 17:41:01.566483+02:00                      650   
 2025-06-19 17:41:22.279200+02:00                      650   
 
 Company Name                     Amalgamated Regional Trading (Art) Holdings Limited  \
 Date                                                                                   
 2025-06-19 17:41:01.566483+02:00                                              17.05    
 2025-06-19 17:41:22.279200+02:00                                              17.05    
 
 Company Name                     Ariston Holdings Limited  \
 Date                                                        
 2025-06-19 17:41:01.566483+02:00                   4.0346   
 2025-06-19 17:41:22.279200+02:00                   4.0346   
 
 Company Name                     British American Tobacco Zimbabwe Limited  \
 Date                                                          

In [9]:
def save_data_with_index_timestamp(open_price: pd.DataFrame, close_price: pd.DataFrame, vol_traded: pd.DataFrame):
    # Extract the index name (timestamp) from the DataFrames
    timestamp = open_price.index[0]
    
    # Create the directory path using the timestamp
    directory_path = f'archive/{timestamp}'
    
    # Create the directory if it doesn't exist
    os.makedirs(directory_path, exist_ok=True)
    
    # Save the DataFrames to JSON files with the timestamped directory
    open_price.to_json(f'{directory_path}/open_price.json', orient='split', date_format='iso')
    close_price.to_json(f'{directory_path}/close_price.json', orient='split', date_format='iso')
    vol_traded.to_json(f'{directory_path}/vol_traded.json', orient='split', date_format='iso')

# Example usage
save_data_with_index_timestamp(open_price, close_price, vol_traded)

In [10]:
open_price

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,Delta Corporation Limited,Ecocash Holdings Zimbabwe Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-19 17:41:01.566483+02:00,650,17.05,4.0346,8010,1800,699.7917,539,164.3457,1338.3775,12,...,5,96.75,172,7,180,3.9975,300,0.0018,10.8,30.0001
2025-06-19 17:41:22.279200+02:00,650,17.05,4.0346,8010,1800,699.7917,539,164.3457,1338.3775,12,...,5,96.75,172,7,180,3.9975,300,0.0018,10.8,30.0001


In [11]:
close_price

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,Delta Corporation Limited,Ecocash Holdings Zimbabwe Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-19 17:41:08.948286+02:00,650,17.05,4.04,8000,1800,699.95,539,164.3553,1338.4551,12.5,...,5.0773,99.1827,172,7,180,3.9853,300,0.0018,10.8,32.4
2025-06-19 17:41:30.372296+02:00,650,17.05,4.04,8000,1800,699.95,539,164.3553,1338.4551,12.5,...,5.0773,99.1827,172,7,180,3.9853,300,0.0018,10.8,32.4


In [12]:
vol_traded

Company Name,Afdis Distillers Limited,Amalgamated Regional Trading (Art) Holdings Limited,Ariston Holdings Limited,British American Tobacco Zimbabwe Limited,Cafca Limited,Cbz Holdings Limited,Cfi Holdings Limited,Dairibord Holdings Limited,Delta Corporation Limited,Ecocash Holdings Zimbabwe Limited,...,Starafricacorporation Limited,Tanganda Tea Company Limited,Tsl Limited,Turnall Holdings Limited,Unifreight Africa Limited,Willdale Limited,Zb Financial Holdings Limited,Zeco Holdings Limited,Zimbabwe Newspapers (1980) Limited,Zimre Holdings Limited
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-19 17:41:16.325772+02:00,0,0,15200,100,0,100,0,12300,83800,700,...,25000,10400,192100,7600,0,8000,0,0,0,100
2025-06-19 17:41:37.580546+02:00,0,0,15200,100,0,100,0,12300,83800,700,...,25000,10400,192100,7600,0,8000,0,0,0,100


In [13]:
# import pandas as pd

# Load JSON data into a DataFrame
# def load_json_data(file_path: str) -> pd.DataFrame:
#     return pd.read_json(file_path, orient='split')

# # Example usage
# open_price_df = load_json_data('open_price.json')
# close_price_df = load_json_data('close_price.json')
# vol_traded_df = load_json_data('vol_traded.json')

# # Display the DataFrames
# open_price_df.head()
# close_price_df.head()
# vol_traded_df.head()

In [14]:
#schedule.every(1).day.do(update_data)

#while True:
   #schedule.run_pending()
   #time.sleep(1)

   #print('jobs:', len(schedule.get_jobs()))